In [32]:
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import randint
from sklearn.metrics import accuracy_score

In [7]:
data=load_wine()

In [11]:
X, y = wine.data, wine.target

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [20]:
param_dist = {
    'max_depth': randint(1, 4),
    'min_samples_split': randint(2, 15),
    'min_samples_leaf': randint(1, 15),
    'max_features': randint(1, X.shape[1] + 1)
}

In [21]:
dt = DecisionTreeClassifier()
random_search = RandomizedSearchCV(dt, param_distributions=param_dist, n_iter=100, cv=5, scoring='accuracy', random_state=42)
random_search.fit(X_train, y_train)

RandomizedSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_iter=100,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000002F5C1B2D908>,
                                        'max_features': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000002F5C1B2E948>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000002F5C1B2DFC8>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000002F5C1B2DA48>},
                   random_state=42, scoring='accuracy')

In [22]:
best_params = random_search.best_params_

In [30]:
best_params

{'max_depth': 3,
 'max_features': 11,
 'min_samples_leaf': 3,
 'min_samples_split': 2}

In [24]:
best_accuracy = random_search.best_score_

In [27]:
dt_clf_tuned = DecisionTreeClassifier(**best_params)

In [28]:
dt_clf_tuned.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=3, max_features=11, min_samples_leaf=3)

In [29]:
y_pred = dt_clf_tuned.predict(X_test)

In [33]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of tuned decision tree: {accuracy:.2f}")

Accuracy of tuned decision tree: 0.94


### Continuing with Random Forest

In [35]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import ShuffleSplit

In [36]:
rf_clf = RandomForestClassifier(n_estimators=10, **best_params)

In [37]:
cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=42)

In [38]:
forest_accuracy = []
for train_index, test_index in cv.split(X_train):
    X_train_sub, X_test_sub = X_train[train_index], X_train[test_index]
    y_train_sub, y_test_sub = y_train[train_index], y_train[test_index]

    rf_clf.fit(X_train_sub, y_train_sub)
    y_pred_sub = rf_clf.predict(X_test_sub)
    accuracy = accuracy_score(y_test_sub, y_pred_sub)
    forest_accuracy.append(accuracy)

In [39]:
avg_forest_accuracy = sum(forest_accuracy) / len(forest_accuracy)
print(f"Average test set accuracy of the random forest: {avg_forest_accuracy:.2f}")

Average test set accuracy of the random forest: 0.92


In [40]:
print(f"Performance improvement: {avg_forest_accuracy - accuracy:.2f}")

Performance improvement: -0.01


We Cant say better or poor. It is almost the Same with a slight improvement in Decision Tree Classifier